In [19]:
import pandas as pd
import sys
import os
import string
import numpy as np
import scipy.stats as stats
import re

In [20]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):    
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [21]:
def calc_mean(list):
   
    mean = np.mean(list)/1000
    return mean

    # std = np.std(list)
    # normal_list = []
    # for l in list:
    #     z = (l-mean)/std
    #     if z <= 1 and z>= -1:
    #         normal_list.append(l)
    #     avg_value = np.mean(normal_list) /1000 
    #     return avg_value 

In [22]:
def merge_results(paths):
    dfs = []
    base_index = 0
    for path in paths:        
        data = pd.read_csv( path, sep=',')
        df_index = [ *range(base_index, base_index + len(data), 1)]
        base_index += len(data)
        data.index=df_index
        dfs.append(data)


    result =  pd.concat(dfs)
    return result


def calc_read_write(header,baselines, languages, tasksets, platforms,seq_rands, nrows, df_run, df_load_to_memory, execution):
   
    df_final = pd.DataFrame(columns = header)    
    for baseline in baselines:
        for language in languages:
            for taskset in tasksets:
                for platform in platforms:
                    for seq_rand in seq_rands:
                        for nrow in nrows:                                                                      
                            df_total =  df_run.loc[(df_run['baseline']==baseline) &
                                                   (df_run["language"]==language) &
                                                   (df_run['taskset']==taskset) &
                                                   (df_run['execution']=='Total') &
                                                   (df_run['platform']==platform) &
                                                   (df_run['seq_rand']==seq_rand) &
                                                   (df_run['nrow']==nrow)]

                            list_total = np.array(df_total['time'].tolist())  

                            df_cpu_io =  df_run.loc[(df_run['baseline']==baseline) &
                                                    (df_run["language"]==language) &
                                                    (df_run['taskset']==taskset) &
                                                    (df_run['execution']==execution) &
                                                    (df_run['platform']==platform) &
                                                    (df_run['seq_rand']==seq_rand) &
                                                    (df_run['nrow']==nrow)]

                            list_cpu_io = np.array(df_cpu_io['time'].tolist())  

                            df_load =  df_load_to_memory.loc[(df_load_to_memory["language"]==language) &
                                                             (df_load_to_memory['taskset']==taskset) &                                                                                                                    
                                                             (df_load_to_memory['platform']==platform) &
                                                             (df_load_to_memory['nrow']==nrow)]

                            list_load = np.array(df_load['time'].tolist())  


                            if len(list_total) > 0:
                                    
                                    time_total = calc_mean(list=list_total)    
                                    time_cpu_io = calc_mean(list=list_cpu_io)
                                    time_load = calc_mean(list=list_load)                                    

                                    base = baseline.replace("FlatBuffersJava", "FlatBufJava")                                    
                                    if execution == "CPU" :
                                        print(f'total={list_total} cpu={list_cpu_io}  load={list_load}  ')
                                        total = time_total - time_load 
                                        io  = time_total - time_cpu_io
                                        cpu  = time_cpu_io - time_load
                                     
                                    elif execution == "IO":                                 
                                        total = time_total
                                        cpu = time_total - time_cpu_io       
                                        io = time_cpu_io

                                    result_total = [base,language,taskset,'Total',platform,seq_rand,nrow,f'{total:.2f}']
                                    result_cpu = [base,language,taskset,'CPU',platform,seq_rand,nrow,f'{cpu:.2f}']
                                    result_io = [base,language,taskset,"IO",platform,seq_rand,nrow,f'{io:.2f}']    

                                    df_final.loc[len(df_final)] = result_total
                                    df_final.loc[len(df_final)] = result_cpu
                                    df_final.loc[len(df_final)] = result_io



  
    return df_final  

In [23]:
def results():
    root_path = '../archive'
    # write_results = [f'{root_path}/read_write/CPP_R13_Experiment1_Write_times.dat', 
    #                  f'{root_path}/read_write/Java_R1_Experiment1_Write_times.dat',
    #                  f'{root_path}/read_write/Java_R2_Experiment1_Write_times.dat',
    #                  f'{root_path}/read_write/Java_R3_Experiment1_Write_times.dat',
    #                  f'{root_path}/read_write/Rust_R13_Experiment1_Write_times.dat']

    # read_results = [f'{root_path}/read_write/CPP_R13_Experiment2_Read_times.dat', 
    #                 f'{root_path}/read_write/Java_R1_Experiment2_Read_times.dat',
    #                 f'{root_path}/read_write/Java_R2_Experiment2_Read_times.dat',
    #                 f'{root_path}/read_write/Java_R3_Experiment2_Read_times.dat',
    #                 f'{root_path}/read_write/Rust_R13_Experiment2_Read_times.dat']                 


   
    # load_to_memory_results = [f'{root_path}/read_write/CPP_R13_Experiment1_LoadToMemory_times.dat',
    #                           f'{root_path}/read_write/Java_R1_Experiment1_LoadToMemory_times.dat',
    #                           f'{root_path}/read_write/Java_R2_Experiment1_LoadToMemory_times.dat',
    #                           f'{root_path}/read_write/Java_R3_Experiment1_LoadToMemory_times.dat',
    #                           f'{root_path}/read_write/Rust_R13_Experiment1_LoadToMemory_times.dat']

    write_results = [f'{root_path}/_results/Experiment1_Write_times.dat']

    read_results = [f'{root_path}/_results/Experiment2_Read_times.dat']                 
   
    load_to_memory_results = [f'{root_path}/_results/Experiment1_LoadToMemory_times.dat']
   
    read_write_header = ["baseline","language","taskset","execution","platform","seq_rand","nrow","time"]   
    load_to_memory_header = ["language","platform","taskset","nrow","time"]

    baselines = ["MessagePackRust"]
    #["DefaultJava","Json+GzipJava","BsonJava","ProtoBufJava","KryoJava","ByteBufferJava","JsonJava","FlatBuffersJava",
    #            "HandcodedCPP","inPlaceCPP","BoostCPP","ProtoBufCPP","BsonCPP","FlatBufCPP",
    #            "JsonRust","BincodeRust","MessagePackRust","BsonRust","FlexBufRust"]    
    
    nrows = [5000000,4000000,3000000,2000000,1000000]

    executions = ["CPU", "IO", "Total"]
    platforms  = ["Parallel" ] #"Single" , 
    seq_rands  = ["Sequential", "Random"]
    languages  = ["Java", "CPP","Rust"]
    tasksets   = [True, False]

    df_write = merge_results(write_results)
    df_read = merge_results(read_results)
    df_load_to_memory = merge_results(load_to_memory_results)
    #print(df_load_to_memory)

    #Parallel,false,5000000,

    df_load =  df_load_to_memory.loc[(df_load_to_memory["language"]=='CPP') &
                                     (df_load_to_memory['taskset']==False) &                                                                                                                    
                                     (df_load_to_memory['platform']=='Parallel') &
                                     (df_load_to_memory['nrow']==5000000)]
    
    list_load = np.array(df_load['time'].tolist())  
    # print(df_load)                                 

    #calc_read_write(header,baselines, languages, tasksets, platforms,seq_rands, nrows, df_run, df_load, execution):
   
    df_write_final = calc_read_write(header=read_write_header, baselines=baselines, languages=languages, tasksets=tasksets, platforms=platforms,
                                      seq_rands= seq_rands, nrows=nrows, df_run=df_write, df_load_to_memory= df_load_to_memory, execution="CPU")

                            
    df_write_final.to_csv("write3.csv", index=False)                  
  


   

In [24]:
results()

total=[20283 20554 20796 20236 20176] cpu=[20714 20619 20348 20413 20476]  load=[20066 20137 20343 20369 20428 20482 20461 20300 20367 20746 20520 20507
 20399 20119 20092 20448 20804 20221 20189 20466 20264 20396 20139 20255]  
